# Senegal Assessment Notebook

Notebook to perform assessment of CHM and Land Cover results over senegal.

## TODO

- Add marker of imagery based on year options
- Add marker of imagery and zoom into overlay raster
- Add marker of imagery and push on and off landcover layers
- Marker with histogram, super-imposing the histograms
- Plot multiple images

## Define data paths

Here we define data path with regexs.

In [1]:
import os
import tempfile
import folium
import numpy as np
import rasterio as rio
import ipywidgets as widgets
import branca.colormap as cm
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
import json

from glob import glob
from folium import plugins
from pyproj import Transformer 
from rasterio.warp import calculate_default_transform, reproject, Resampling

os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = \
    f"{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}"

from localtileserver import get_folium_tile_layer, TileClient

In [2]:
data_bands = [5, 2, 7]
data_dir = '/adapt/nobackup/projects/3sl/data/Tappan/*_data.tif'
landcover_dir = '/adapt/nobackup/projects/ilab/projects/Senegal/3sl/products/land_cover/dev/trees.v2/Tappan/*.tif'
chm_dir = '/adapt/nobackup/projects/ilab/projects/Senegal/CNN_CHM/v4/output/*.tif'

Then, we generate lists out of the files available on the dirs.

In [3]:
data_filenames = sorted(glob(data_dir))
landcover_filenames = sorted(glob(landcover_dir))
chm_filenames = sorted(glob(chm_dir))
print(len(data_filenames), len(landcover_filenames), len(chm_filenames))

272 272 272


## Define basemaps for later use

Here we define basemaps for use with folium on the backend.

In [4]:
basemaps = {
       'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = False,
        control = True
       ),
        'basemap_gray': folium.TileLayer(
            tiles="http://services.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}",
            opacity=1,
            name="World gray basemap",
            attr="ESRI",
            overlay=False
        ),
        'Imagery': folium.TileLayer(
            tiles='https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            opacity=1,
            name="World Imagery",
            attr="ESRI",
            overlay=False
        ),
        'ESRINatGeo': folium.TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}',
            opacity=1,
            name='ESRI NatGeo',
            attr='ESRI',
            overlay=False
        )
}

## Define Generic Help Functions

In this section we transform the data and plot.

In [ ]:
def reproject_raster(input_tiff, dst_crs=f"EPSG:3857"):
    """
    Reproject raster.
    """
    # set out path
    out_path_rproj = os.path.join(
        tempfile.gettempdir(),
        input_tiff.split('/')[-1].replace('.tif','.reprojected.tif')
    )

    with rio.open(input_tiff) as src:
    
        # get src bounds and transform
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        print(width, height)
        kwargs = src.meta.copy()
        kwargs.update(
            {
                'crs': dst_crs,
                'transform': transform,
                'width': width,
                'height': height
        })
    
        # reproject and write to file
        with rio.open(out_path_rproj, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(source=rio.band(src, i),
                      destination=rio.band(dst, i),
                      src_transform=src.transform,
                      src_crs=src.crs,
                      dst_transform=transform,
                      dst_crs=dst_crs,
                      resampling=Resampling.nearest)
    
    return out_path_rproj

def get_bounds(tiff_3857):
    with rio.open(tiff_3857) as src:
        src_crs = src.crs['init'].upper()
        min_lon, min_lat, max_lon, max_lat = src.bounds
    bounds_orig = [[min_lat, min_lon], [max_lat, max_lon]]
    bounds = []
    dst_crs = 'EPSG:4326'
    for item in bounds_orig:   
        #converting to lat/lon
        lat = item[0]
        lon = item[1]
        proj = Transformer.from_crs(
            int(src_crs.split(":")[1]), int(dst_crs.split(":")[1]), always_xy=True)
        lon_n, lat_n = proj.transform(lon, lat)
        bounds.append([lat_n, lon_n])
    center_lon = bounds[0][1] + (bounds[1][1] - bounds[0][1])/2
    center_lat = bounds[0][0] + (bounds[1][0] - bounds[0][0])/2
    return {'bounds': bounds, 'center': (center_lon, center_lat)}

def open_and_get_band(file_name, band_num=1):
    with rio.open(file_name) as data:
        b = data.read(band_num)
    return b

def get_overlay(band, meta_dict, name, opacity=1.0, show=True, colormap=None):
    if colormap is not None:
        return folium.raster_layers.ImageOverlay(
            band, bounds=meta_dict['bounds'], 
            name=name, opacity=opacity, show=show, colormap=plt.get_cmap('YlGn'))
    else:
        return folium.raster_layers.ImageOverlay(
            band, bounds=meta_dict['bounds'], 
            name=name, opacity=opacity, show=show)

def cleanup(filename):
    if os.path.exists(filename):
        os.remove(filename)
    else:
        print('No file: {} exists.'.format(filename))

## Modify Data and Plot

In this section we transform the data and plot.

In [ ]:
#data_filenames_3857 = []
#landcover_filenames_3857 = []
#chm_filenames_3857 = []
#for d, l, c in zip(data_filenames, landcover_filenames, chm_filenames):    
#    data_filenames_3857.append(reproject_raster(d))
#    landcover_filenames_3857.append(reproject_raster(l))
#    chm_filenames_3857.append(reproject_raster(c))
data_filenames_3857 = reproject_raster(data_filenames[0])
landcover_filenames_3857 = reproject_raster(landcover_filenames[0])
chm_filenames_3857 = reproject_raster(chm_filenames[0])

Then we take the temporary bounds.

In [ ]:
data_filenames_bounds = get_bounds(data_filenames_3857)
landcover_filenames_bounds = get_bounds(landcover_filenames_3857)
chm_filenames_bounds = get_bounds(chm_filenames_3857)
print(data_filenames_bounds, landcover_filenames_bounds, chm_filenames_bounds)

Then we open the data rasters.

In [ ]:
data_filenames_mask = open_and_get_band(data_filenames_3857, data_bands)
landcover_filenames_mask = open_and_get_band(landcover_filenames_3857, 1)
chm_filenames_mask = open_and_get_band(chm_filenames_3857, 1)
print(data_filenames_mask.max(), landcover_filenames_mask.max(), chm_filenames_mask.max())

In [ ]:
zeros = np.zeros_like(landcover_filenames_mask)
data_mask_rgb = np.moveaxis(data_filenames_mask, 0, -1)
landcover_mask_rgb = np.dstack((zeros, landcover_filenames_mask, zeros))
#chm_mask_rgb = chm_filenames_mask  # np.dstack((zeros, zeros, chm_filenames_mask))
chm_filenames_mask[chm_filenames_mask < 0] = 0
chm_mask_rgb = chm_filenames_mask
print(data_mask_rgb.max(), landcover_mask_rgb.max(), chm_mask_rgb.max())

In [ ]:
# histogram

testing_histogram = chm_mask_rgb
testing_histogram = testing_histogram.flatten()
testing_histogram = testing_histogram[testing_histogram != 0][:5000]
testing_histogram.shape

# Generating Data
source = pd.DataFrame({
    'CHM': testing_histogram,
})

chart = alt.Chart(source).transform_fold(
    ['CHM'],
    as_=['Experiment', 'Measurement']
).mark_bar(
    opacity=0.3,
    binSpacing=0
).encode(
    alt.X('Measurement:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('count()', stack=None),
    #alt.Color('Experiment:N')
)
chart_2 = json.loads(chart.to_json())
chart

Here we create the general object for the folium map.

In [ ]:
# Map the Layers
map_figure = folium.Figure(width=1000, height=400)
folium_map = folium.Map(
    [landcover_filenames_bounds['center'][1], landcover_filenames_bounds['center'][0]],
    zoom_start=6,
    control_scale=True,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google'
)

folium_map.add_child(
    get_overlay(
        data_mask_rgb, data_filenames_bounds,
        'data layer')#, opacity=0.6)
)
folium_map.add_child(
    get_overlay(
        landcover_mask_rgb, landcover_filenames_bounds,
        'land cover mask', opacity=0.6, show=False)
)

pal_height_cmap = cm.LinearColormap(
    colors=['black','#636363','#fc8d59','#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850'], 
    vmin=0, vmax=50
)

folium_map.add_child(
    get_overlay(
        chm_mask_rgb, chm_filenames_bounds,
        'chm mask', opacity=1.0, show=False, colormap=pal_height_cmap
    )
)

popup = folium.Popup()
folium.VegaLite(json.loads(chart.to_json()), width="100%", height="100%").add_to(popup)

folium.Marker(
    [
        landcover_filenames_bounds['center'][1],
        landcover_filenames_bounds['center'][0]
    ],
    popup=popup
).add_to(folium_map)

folium_map.add_child(plugins.Fullscreen())
folium_map.add_child(plugins.Geocoder())
folium_map.add_child(plugins.MousePosition())
folium_map.add_child(folium.LayerControl())

In [ ]:
data_filenames = sorted(glob(data_dir))
landcover_filenames = sorted(glob(landcover_dir))
chm_filenames = sorted(glob(chm_dir))
print(len(data_filenames), len(landcover_filenames), len(chm_filenames))

# Map the Layers
map_figure = folium.Figure(width=1000, height=400)
folium_map = folium.Map(
    #[data_filenames_bounds['center'][1], data_filenames_bounds['center'][0]],
    #zoom_start=6,
    #control_scale=True,
    #tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    #attr='Google'
    location=client.center(), zoom_start=16
)

#folium_map.add_child(
#    #get_overlay(
#    #    data_mask_rgb, data_filenames_bounds,
#    #    'data layer')#, opacity=0.6)
#    get_folium_tile_layer(TileClient(data_filenames[0]), attr='DataLayer')
#)

folium_map.add_child(
    #get_overlay(
    #    data_mask_rgb, data_filenames_bounds,
    #    'data layer')#, opacity=0.6)
    get_folium_tile_layer(TileClient(data_filenames[0]), attr='DataLayer')
)



#folium_map.add_child(
#    get_overlay(
#        landcover_mask_rgb, landcover_filenames_bounds,
#        'land cover mask', opacity=0.6, show=False)
#)

#pal_height_cmap = cm.LinearColormap(
#    colors=['black','#636363','#fc8d59','#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850'], 
#    vmin=0, vmax=50
#)

#folium_map.add_child(
#    get_overlay(
#        chm_mask_rgb, chm_filenames_bounds,
#        'chm mask', opacity=1.0, show=False, colormap=pal_height_cmap
#    )
#)

#popup = folium.Popup()
#folium.VegaLite(json.loads(chart.to_json()), width="100%", height="100%").add_to(popup)

#folium.Marker(
#    [
#        data_filenames_bounds['center'][1],
#        data_filenames_bounds['center'][0]
#    ],
#).add_to(folium_map)

#folium_map.add_child(plugins.Fullscreen())
#folium_map.add_child(plugins.Geocoder())
#folium_map.add_child(plugins.MousePosition())
#folium_map.add_child(folium.LayerControl())

In [6]:
# First, create a tile server from local raster file
client = TileClient('/adapt/nobackup/projects/ilab/projects/Senegal/3sl/products/land_cover/dev/experiments/otcb.all/ETZ/WV03_20170207_M1BS_10400100271ED400-toa.otcb.tif')
#client = TileClient('/adapt/nobackup/projects/3sl/data/Tappan/Tappan24_WV03_20171001_M1BS_1040010034837400_data.tif')
# client = examples.get_oam2()  # use example data

# Create folium tile layer from that server
t = get_folium_tile_layer(client)

m = folium.Map(location=client.center(), zoom_start=16)
m.add_child(t)
m

In [ ]:
#chm_mask_rgb.shape

In [ ]:
#from localtileserver import get_leaflet_tile_layer, TileClient
#from ipyleaflet import Map
# First, create a tile server from local raster file
#client = TileClient('/adapt/nobackup/projects/ilab/projects/Senegal/3sl/products/land_cover/dev/experiments/otcb.all/ETZ/WV03_20170207_M1BS_10400100271ED400-toa.otcb.tif')
# Create ipyleaflet tile layer from that server
#t = get_leaflet_tile_layer(client)
#m = Map(center=client.center(), zoom=client.default_zoom)
#m.add_layer(t)
#m

In [ ]:
#from localtileserver import get_folium_tile_layer, TileClient, examples
#from folium import Map

# First, create a tile server from local raster file
# client = TileClient('/adapt/nobackup/projects/ilab/projects/Senegal/3sl/products/land_cover/dev/experiments/otcb.all/ETZ/WV03_20170207_M1BS_10400100271ED400-toa.otcb.tif')
#client = TileClient('/adapt/nobackup/projects/3sl/data/Tappan/Tappan24_WV03_20171001_M1BS_1040010034837400_data.tif', port=33000)
# client = examples.get_oam2()  # use example data

# Create folium tile layer from that server
#t = get_folium_tile_layer(client)

#m = Map(location=client.center(), zoom_start=16)
#m.add_child(t)
#m

In [ ]:
#import localtileserver
#print(localtileserver.Report())

In [ ]:
#import os
#os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = \
#    f"{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}"
#LOCALTILESERVER_CLIENT_PREFIX='proxy/{port}'
#LOCALTILESERVER_CLIENT_HOST=127.0.0.1
#LOCALTILESERVER_CLIENT_PORT=8888

In [ ]:
#from localtileserver import get_folium_tile_layer, TileClient, examples
#from folium import Map

# First, create a tile server from local raster file
#client = TileClient('/adapt/nobackup/projects/ilab/projects/Senegal/3sl/products/land_cover/dev/experiments/otcb.all/ETZ/WV03_20170207_M1BS_10400100271ED400-toa.otcb.tif')
#client = TileClient('/adapt/nobackup/projects/3sl/data/Tappan/Tappan24_WV03_20171001_M1BS_1040010034837400_data.tif')
# client = examples.get_oam2()  # use example data

# Create folium tile layer from that server
#t = get_folium_tile_layer(client)
#m = Map(location=client.center(), zoom_start=16)
#m.add_child(t)
#m